<a href="https://colab.research.google.com/github/fengfrankgthb/BUS-41204/blob/main/QQQ/QQQ_Paired_Strategy1" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Paired Strategy Project With QQQ

In [1]:
# Read the CSV file directly from the /raw.githubusercontent.com/ URL
import pandas as pd
url = "https://raw.githubusercontent.com/fengfrankgthb/BUS-41204/main/QQQ/QQQ_history_990310_250505.csv"
df = pd.read_csv(url)
df[2:5] # Display 3rd-5th rows of data. Index=2 is 3rd row included, whereas index=5 is 6th row excluded.

,Date,Open,High,Low,Close,Volume,Dividends,Stock Splits,Capital Gains
2,1999-03-12 00:00:00-05:00,43.335818,43.362307,42.090840,42.435196,8743600,0.0,0.0,0.0
3,1999-03-15 00:00:00-05:00,42.753072,43.706672,42.302761,43.653694,6369000,0.0,0.0,0.0
4,1999-03-16 00:00:00-05:00,43.839095,44.209939,43.362295,44.024517,4905800,0.0,0.0,0.0
